In [1]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D, Input
from tensorflow.keras import Model, mixed_precision

import os

os.chdir('..')

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [3]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [4]:
from sam import sam_train_step

class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, dtype='float32')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)
  
  def train_step(self, data):
    return sam_train_step(self, data)

# Create an instance of the model
model = MyModel()

In [5]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [6]:
model.fit(train_ds, epochs=5,
          validation_data=test_ds)

Epoch 1/5
1875/1875 [==============================] - 85s 45ms/step - loss: 0.3051 - accuracy: 0.8975 - val_loss: 0.0625 - val_accuracy: 0.9790
Epoch 2/5
1875/1875 [==============================] - 84s 45ms/step - loss: 0.0700 - accuracy: 0.9740 - val_loss: 0.0502 - val_accuracy: 0.9831
Epoch 3/5
1875/1875 [==============================] - 84s 45ms/step - loss: 0.0472 - accuracy: 0.9813 - val_loss: 0.0544 - val_accuracy: 0.9835
Epoch 4/5
1875/1875 [==============================] - 85s 46ms/step - loss: 0.0326 - accuracy: 0.9867 - val_loss: 0.0556 - val_accuracy: 0.9843
Epoch 5/5
1875/1875 [==============================] - 82s 44ms/step - loss: 0.0234 - accuracy: 0.9906 - val_loss: 0.0617 - val_accuracy: 0.9852
